Итоговое задание по 3-му модулю.

Часть 1. Разработка инструмента для анализа киберугроз с использованием VirusTotal API 
Исходные данные:
 Архив ZIP с файлами для анализа. Файл protected_archive.zip.
 Пароль: netology
 Этапы выполнения задания (часть 1)
 Этап 1. Распаковка архива. Используя Python, распакуйте предоставленный архив и извлеките файлы.
 Этап 2. Анализ файлов через VirusTotal API. Отправьте файлы на анализ, используя ваш персональный API-ключ VirusTotal.
 Этап 3. Обработка результатов сканирования. Проанализируйте ответы от VirusTotal, собирая данные о детектировании угроз антивирусами.
 Этап 4. Подготовка отчёта. Составьте отчёт со статистикой результатов сканирования. Включите в отчёт код скрипта и результат его вывода в виде скриншота (JPG, PNG). Приведите список антивирусов, которые обнаружили угрозы, в формате: Detected, ALYac, Kaspersky. Сравните результаты с заданным списком антивирусов и песочниц. Укажите, какие из указанных антивирусов (Fortinet, McAfee, Yandex, Sophos) детектировали угрозу, а какие нет.
 Дополнительные задачи:
 - Если доступен отчёт VirusTotal Sandbox о поведении вредоноса, проанализируйте его и включите в свой отчёт ключевые моменты из него.
 - Выведитесписок доменов и IP-адресов, с которыми вредонос общается, (для блокировки) и описание поведения (Behavior) от VirusTotal Sandbox, если оно  доступно.

In [7]:
import requests

API_KEY = "0d97c91facc32d469c4b0499baaca1cc456dd2f872ab73c8d91eaf1961ada2a2"
FILE_NAME = "protected_archive.zip"
PASSWD = "netology"

#функция загружает файл на VT
def upload_archive():
    url = "https://www.virustotal.com/api/v3/files"
    files = { "file": (FILE_NAME, open(FILE_NAME, "rb"), "application/x-zip-compressed") }
    payload = { "password": PASSWD }
    headers = {
        "accept": "application/json",
        "x-apikey": API_KEY
    }

    response = requests.post(url, data=payload, files=files, headers=headers)

    if response.status_code == 200:
        try:
            json_response = response.json()
            scan_id = json_response['data']['id']
            print("Архив отправлен на Virus Total!")
            print(f"ID сканирования: {scan_id}")
            return scan_id
        except(KeyError, ValueError):
            print (f"Ошибка отправки файла {response.status_code} {respresponse.text}")
    else:
        print(f"Ошибка при получении ответа: {response.status_code} - {response.text}")
        return None


# функция запрашивает результаты обработки на VT по ID
def get_analysis_archive(scan_id):
    scan_id = scan_id
    url = f"https://www.virustotal.com/api/v3/analyses/{scan_id}"
    headers = {
        "accept": "application/json",
        "x-apikey": API_KEY
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        try:
            json_response = response.json()
            malicious_result = json_response['data']['attributes']['stats']['malicious']
            undetected_result = json_response['data']['attributes']['stats']['undetected']
            print("Результаты анализа:")
            print(f"Вердикт вредоносный - {malicious_result} антивирус(ов)")
            print(f"Вердикт необнаружено - {undetected_result} антивирус(ов)")
            if malicious_result >= 1:
                print("Файл опасен!")
                #выписывает все антивирусы, которые задетектировали угрозу
                antivirus_detected = []
                for antivirus, result in json_response['data']['attributes']['results'].items():
                    if result['category'] == 'malicious':
                        antivirus_detected.append(antivirus)
                print("Обнаружено следующим антиврусами:")
                print(antivirus_detected)
            else:
               print("Файл не представляет угрозы") 
        except (KeyError, ValueError):
            print(f"Ошибка при разборе ответа JSON: {response.text}")
            return None
    else:
        print(f"Ошибка при получении результатов анализа: {response.status_code} - {response.text}")
        return None


def main():
    scan_id = upload_archive()
    get_analysis_archive(scan_id)
    print("-"*30)
    

if __name__ == "__main__":
    main()


Архив отправлен на Virus Total!
ID сканирования: NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjgxNzg5OQ==
Результаты анализа:
Вердикт вредоносный - 26 антивирус(ов)
Вердикт необнаружено - 36 антивирус(ов)
Файл опасен!
Обнаружено следующим антиврусами:
['Lionic', 'MicroWorld-eScan', 'ClamAV', 'CTX', 'Sangfor', 'Symantec', 'ESET-NOD32', 'Avast', 'Kaspersky', 'BitDefender', 'Emsisoft', 'DrWeb', 'VIPRE', 'Ikarus', 'FireEye', 'Varist', 'Fortinet', 'Kingsoft', 'Arcabit', 'ViRobot', 'Microsoft', 'Google', 'ALYac', 'GData', 'AVG', 'alibabacloud']
------------------------------
